### Notebook Overview:
* The following notebook is a condensed version demonstrating the core functionality of the VAE and GA modules. This notebook trains 1 VAE model and deploys the resultant low-dimensional representation into a genetic algorithm optimisation framework to maximise the compactness metric.
* For a complete functionality, we reccomend cloning the GitHub repo and running the examples inside ```/examples/vae_examples/``` and ```/examples/ga_examples/```

### Core Packages:

* ```vae``` - ```/content/latentoptim/vae``` : For building and training the VAE models.
* ```data``` - ```/content/latentoptim/data``` : For creating geometric shape training data.
* ```utils``` - ```/content/latentoptim/utils``` : Supporting functionality (e.g, visualising latent distributions & generated 
shapes.)

* ```pyga``` - https://github.com/mark-hobbs/pyga : Abstracted genetic algorithm for easy integration with numerical optimisation objectives.

### Import GitHub Repo and Install Package on Colab:

In [ ]:
!git clone https://github.com/jhell1717/latentoptim.git
!pip install git+https://github.com/jhell1717/latentoptim.git

### Imports
* Import standard packages

In [ ]:
import os
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt
import torch

# Our custom packages.
import pyga
import vae
import data
import utils

if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

### Config:
* ```shape_resolution``` = Number of nodes representing the random shapes.
* ```num_shapes``` = Number of shapes to generate for training the VAE.
* ```latent_dimensions``` = Number of latent dimensions in the VAE used to represent the high dimensional shapes. Smaller = more compression, higher = more expressiveness.
* ```learning_rate``` = Used to specify the step size taken during VAE training in back propogation.
* ```batch_size``` = Number of training examples to pass through neural network at once. Increase or decrease depending on available memory.

In [ ]:
shape_resolution = 200
num_shapes = 10000
latent_dimensions = 3

learning_rate = 1e-3
batch_size = 512

# 1) Train VAE Model:

### Generate Random Shape Dataset:
* User specifies resolution with ```resolution``` variable in ```Generator()``` class.
* User specifies number of shapes to generate with ```num_shapes``` in ```Generator()``` class.
* ```Generator``` class is used to create random shape objects.
* Generated shapes will be stored as a .pkl file here: ```/content/latentoptim/examples/vae_examples/colab/demo_shapes.pkl```

In [ ]:
base_dir = r'/content/latentoptim/examples/vae_examples/colab'
# Creates shape random shape data.
shape_data = data.Generator(resolution=shape_resolution,num_shapes=num_shapes).generate_shapes()

file_path = os.path.join(base_dir,'demo_shapes.pkl'),

# Save shape population as .pkl
with open(file_path[0], "wb") as f:
    pickle.dump(shape_data, f)

dataset = vae.ShapeData(shape_data) # Create dataset object

### Visualise a random generated shape from the dataset:
* Random shapes are sampled from the generated population.
* All shapes are examples from the defined categories specified in the ```Generator``` class.

***Shape Representation***
* The shapes are represented as a series of nodes on an x,y plane.
* High resolution shape representations might include, 200+ nodes.
* To optimise shapes in this representation, we require adjusting all 200 nodes to change the resultant geometry.
* In realising a low-dimensional representation, defined as the latent representation, we can reduce the dimensionality of the design space.


In [ ]:
random_shape_id = np.random.choice(range(0,len(dataset.shapes))) # Select random test index from dataset
dataset.shapes[random_shape_id].plot() # Visualise shape. 

### Build VAE Model:
* User specifies ```input_size```. e.g., 200 nodes x,y = 200*2
* User specifies dimension of latent space with ```latent_dim``` variable.
* ```VAE``` class is used to build the VAE architecture.

In [ ]:
model = vae.VAE(input_size=shape_resolution*2,latent_dim=latent_dimensions) # Create model object
model.to(device) # Assign model to GPU if available.

### Train VAE Model:
* Train the VAE model defined as ```model``` above.
* User specifies epochs, batch size and frequency of training checkpoints.
* Change the ```model_name``` variable to denote a new model.
* The model will be saved and the loss plot stored in ```/content/latentoptim/examples/vae_examples/colab/example_model```

In [ ]:
trainer = vae.Trainer(dataset, model, base_dir=base_dir,
                      trained_data=os.path.join(base_dir,'demo_shapes.pkl'), model_name='example_model2', batch_size=batch_size,lr=learning_rate)

# Train model
trainer.train_model(epochs=50,checkpoint_interval=20)

### Specify Model Details
* In this example, we test only 1 model. 
* For additional models, we can add additional models into the array ```models``` and specify their corresponding latent dimensions in ```latent_dims```

In [ ]:
models = [model.to('cpu')] # List of trained models to sample and generate designs.
latent_dims = [latent_dimensions] # List of dimensions associated with each latent model.

### Sample Trained VAE Model:
* For each combination of the latent variables (if > 1), we generate the latent space for shapes samples between latent values $[-3,+3]$
* Each 2D visualisation of latent dimension pairings are stored here: ```/content/latent_space_plots```
* We fix all but the visualised latent dimensions to values of $0.0$

In [ ]:
utils.plot_all_latent_combinations(models,latent_dims,vae_metrics=vae.Metrics,shapes_path=os.path.join(base_dir,'demo_shapes.pkl'))

# 2) Genetic Algorithm Optimisation:

In [ ]:
utils.ShapeVAE.set_model(model) # Set trained model to model in GPR workflow.

### Generate Shape Samples from VAE:

In [ ]:
population_size = 20 # GA initial population size. 

individuals = [utils.ShapeVAE(np.random.uniform(-3, 3, size=3)) for _ in range(population_size)] # Sample from random distribution.

population = pyga.Population(individuals) # Create population with generated individuals.

### Initial Population:
* Before optimisation, the initial population is simply shapes generated by sampling random latent values and reconstructing these latent representations via the decoder.

In [ ]:
population.plot() # View samples of initial population

### Genetic Algorithm Settings:
* ```num_generations``` - Specifies how many evolutions of the GA to run.
* ```num_parents``` - At each evolution, how many individuals to consider for crossover & mutation.
* ```mutation_probability``` - The chance of genes inside the individuals to be mutated.

In [ ]:
ga = pyga.GeneticAlgorithm(population, 
                           num_generations=100, 
                           num_parents=4, 
                           mutation_probability=0.5, 
                           animate=False)

In [ ]:
ga.evolve() # Start GA optimisation

### Fitness Score Tracking
* The maximum fitness value of all sampled in the population is plotted for each generation.
* If the fitness curve has plateaued, the shapes are no longer being optimised.

In [ ]:
ga.plot_fitness()
plt.title(f'Final Fitness Score = {ga.fitness[-1]:.3f}')
plt.tight_layout()

### Visualise final population:
* After all generations of the GA have completed, we visualise final population of shapes.
* If the GA has been successful, shapes should appear more circular. This corresponds to a higher fitness value (see Fitness curve).

In [ ]:
population.plot()